In [ ]:
import pystac_client
import folium
from odc import stac as odc_stac
from ascat.read_native.ragged_array_ts import CellFileCollection
import xarray as xr
from pathlib import Path
import numpy as np
from pyproj import Transformer
import matplotlib.pyplot as plt
from scipy.spatial import cKDTree
import yaml
import holoviews as hv
import dask
import pandas as pd

dask.config.set(**{"array.slicing.split_large_chunks": True})

from dask.distributed import Client

hv.extension("bokeh")
import hvplot.xarray

import warnings

with warnings.catch_warnings():
    warnings.simplefilter("ignore", UserWarning)
    # h121_ds = h121_ds.set_index(time="time")

client = Client()

In [ ]:
paths = yaml.safe_load(Path("../paths.yml").read_text())
root: Path = Path(paths["linux"]).expanduser()
cell_source: Path = root / "datasets/scat_ard/ascat_ssm_cdr_12.5km_h121"
assert cell_source.exists()

### Loading Sentinel - 1 Sigma Nought Data from EODC STAC Catalogue

Setting time and area of interest.

In [ ]:
time_range = "2021-06-01/2021-09-30"

latmin, latmax = 47, 48  # South to North
lonmin, lonmax = 15, 16.5  # West to East

bounding_box = [lonmin, latmin, lonmax, latmax]

Loading the metadata with STAC search engine.

Let's plot thumbnail of the loaded items for this area and those dates.

Now, let's load the data with `odc_stac`.

### Loading EUMETSAT H SAF ASCAT H121 data from local TU Wien directory

Let's extract `gpis` - grid point indices from read data.

In [ ]:
h121_reader = CellFileCollection.from_product_id(cell_source, "H121_V1.0")
gpis, lons, lats, cells = h121_reader.grid.get_grid_points()

Now, we will use specified bounding box that was used to filter out EODC's Sentinel-1 data to filter out grid point indices that are inside this area.

In [ ]:
h121_reader.grid.gpi2cell(h121_reader.grid.get_bbox_grid_points(bounding_box[1], bounding_box[3],bounding_box[0], bounding_box[2]))

In [ ]:
ascat_ds = h121_reader.read(cell=1431)

In [ ]:
ascat_ds.load()

In [ ]:
start_time, end_time = time_range.split("/")


In [ ]:
if "time" not in ascat_ds.indexes:
    ascat_ds = ascat_ds.set_xindex("time")

ascat_ds = ascat_ds.sortby("time").sel(time=slice(start_time, end_time))
ascat_ds

In [ ]:
tdss = h121_reader.read(cell=1431)
tdss.load()

if "time" not in tdss.indexes:
    tdss = tdss.set_xindex("time")

tdss = tdss.sortby("time").sel(time=slice(start_time, end_time))
tdss

In [ ]:
lat_mask = (ascat_ds.lat >= latmin) & (ascat_ds.lat <= latmax)
lon_mask = (ascat_ds.lon >= lonmin) & (ascat_ds.lon <= lonmax)
spatial_mask = lat_mask & lon_mask

ascat_ds = ascat_ds.where(spatial_mask, drop=True)
ascat_ds

In [ ]:
ascat_ds.sat_id.min()

Finally, let's plot the results to see how the grid points look like and what is their spatial resolution.

In [ ]:
lats = ascat_ds["lat"].values
lons = ascat_ds["lon"].values
ids = ascat_ds["location_id"].values

center = [lats.mean(), lons.mean()]
map = folium.Map(location=center, zoom_start=8)

folium.Rectangle(
    bounds=[[latmin, lonmin], [latmax, lonmax]],
    color="blue",
    fill=True,
    fill_opacity=0.1,
    weight=2,
    popup="Area of Interest",
).add_to(map)

for lat, lon, loc_id in zip(lats, lons, ids):
    folium.CircleMarker(
        location=[lat, lon],
        radius=5,
        popup=f"ID: {loc_id}",
        color="blue",
        fill=True,
        fill_color="blue",
    ).add_to(map)

map


In [ ]:
eodc_catalog = pystac_client.Client.open("https://stac.eodc.eu/api/v1")
colllection_id = "SENTINEL1_SIG0_20M"
collection = eodc_catalog.get_collection(colllection_id)

In [ ]:
search = eodc_catalog.search(
    collections=colllection_id,
    bbox=bounding_box,
    datetime=time_range,
)
items_eodc = search.item_collection()

In [ ]:
map = folium.Map(
    location=[(latmin + latmax) / 2, (lonmin + lonmax) / 2],
    zoom_start=7,
    zoom_control=False,
    scrollWheelZoom=False,
    dragging=False,
)

folium.Rectangle(
    bounds=[[latmin, lonmin], [latmax, lonmax]],
    color="blue",
    fill=True,
    fill_opacity=0.1,
    weight=2,
    popup="Area of Interest",
).add_to(map)

for item in items_eodc:
    # url leading to display of an item, can also be used as hyperlink
    image_url = item.assets["thumbnail"].href
    bounds = item.bbox
    folium.raster_layers.ImageOverlay(
        image=image_url,
        bounds=[[bounds[1], bounds[0]], [bounds[3], bounds[2]]],
    ).add_to(map)

folium.LayerControl().add_to(map)

map

In [ ]:
sig0_dc = odc_stac.load(
    items_eodc,
    crs="EPSG:4326",
    resolution=0.0002,  # ~20 meters
    bbox=bounding_box,
    chunks={"time": 1, "longitude": 1000, "latitude": 1000},
    resampling="bilinear",
)

sig0_dc

In [ ]:
item = items_eodc[0]
scale = item.assets["VV"].extra_fields.get("raster:bands")[0]["scale"]
nodata = item.assets["VV"].extra_fields.get("raster:bands")[0]["nodata"]
sig0_dc = sig0_dc.where(sig0_dc != nodata) / scale
# sig0_dc = sig0_dc.interpolate_na(dim="time",
#                                  method="nearest",
#                                  use_coordinate="time",
#                                  max_gap=pd.Timedelta("6h"))
# sig0_dc = sig0_dc.ffill("time")
sig0_dc

In [ ]:
dummy = xr.ones_like(sig0_dc.VH)
dummy = dummy * (dummy.time - dummy.time.min()).astype(int)
dummy
sig0_dc["dummy"] = dummy

In [ ]:
sig0_dc.VV.isel(time=5)[::20, ::20].plot(cmap="viridis", robust=True)

In [ ]:
ascat_ds

In [ ]:
# obs_gpis = tdss["location_id"][tdss["locationIndex"]]
# obs_gpis

In [ ]:
# sig0_regridded = sig0_dc.sel(longitude=obs_gpis.lon, latitude=obs_gpis.lat, time=tdss["time"], method="nearest")  # KERNEL CRASHES
# sig0_regridded

In [ ]:
# sig0_regridded.sel(obs = (obs_gpis == 1205759)).VV.plot()

In [ ]:
sig0_regridded = sig0_dc.sel(longitude=ascat_ds.lon, latitude=ascat_ds.lat, method="nearest")
sig0_regridded["location_id"] = ascat_ds.location_id
regridded = sig0_regridded.set_coords("location_id")
regridded

In [ ]:
time_first = sig0_dc.sel(time=ascat_ds.time, method="nearest")
time_first

In [ ]:
pixel = sig0_dc.isel(latitude=2500, longitude=3500)
pixel = pixel.where(pixel.notnull(), 0)
pixel.VV.plot.scatter(x="time")

In [ ]:
pixel.sel(time=np.datetime64("2021-09-15"), method="nearest").VV.values

In [ ]:
sig0_dc.time

In [ ]:
regridded.dummy.isel(locations=1).plot()

In [ ]:
if len(ascat_ds.time) != len(sig0_regridded.time):
    sig0_regridded = sig0_regridded.sel(time=ascat_ds["time"], method="nearest")

sig0_regridded

In [ ]:
sig0_regridded.VV.isel(locations=1).plot()

In [ ]:
sig0_regridded.VV.isel(locations=2).plot.scatter(x="obs", y="VV")